In [1]:
# Decoder GPT
import torch
import torch.nn as nn
from torch.nn import functional as F

device ='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 64
batch_size = 28
max_iters = 3000
learning_rate = 3e-3
eval_iters = 250
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2

cuda


In [2]:
chars = ""
with open('wizard of oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))
    
vocab_size = len(chars)

In [3]:
#Character level tokeniser

string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print('ix:', ix)
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y 

In [5]:
class Head(nn.Module):
    """ single head of attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B, T, C = x.shape
        k = self.key(x) # (B,T,head_size)
        q = self.query(x) #(B,T,hs)
        # compute attention scores
        weight = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # output (B,T,T)
        
        # Mask tokens and makes them appear at each step
        weight = weight.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B,T,T)
        weight = F.softmax(weight, dim=-1)
        weight = self.dropout(weight)

        v = self.value(x)
        out = weight @ v
        return out # (B,T,hs)
        
        

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parrallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    """ simple linear layer followed by a non-linearity"""
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout) # Prevent overfitting, by dropping dropout % of our neurons
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class GPTLanguageModel(nn.Module): 
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # Helps for convergence of the model
        self.lm_head = nn.Linear(n_embd, vocab_size) #Transform so softmax's computable
        self.apply(self._init_weights)

    # Weight initialization
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None: 
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)


    def forward(self, index, targets=None):
        B, T = index.shape
        
        # Index and targets are both (B, T) tensors of int
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) #(T, C)
        x = tok_emb + pos_emb #(B, T, C)
        x = self.blocks(x) #(B, T, C)
        x = self.ln_f(x) #(B, T, C)
        logits = self.lm_head(x) #(B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape #batch, time, channels
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
        
    def generate(self, index, max_new_tokens):
        # Index is (B, T) array of indices int the current context
        
        for _ in range(max_new_tokens):
            # Get the preditions
            index_cond = index[:, -block_size:]
            logits, loss = self.forward(index_cond)

            # Focus only on the last time step
            logits = logits[:, -1, :] # Becomes (B, C)
    
            # Apply softmax to get the probabilities
            proba = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the distribution
            index_next = torch.multinomial(proba, num_samples=1) # (B, 1)
            # Append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=-1) #(B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)

In [6]:
@torch.no_grad() #decorator reduce computation 
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, y = get_batch(split)
            logits, loss = model(X, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
optimizer= torch.optim.AdamW(model.parameters(), lr = learning_rate) 

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses['train']:.3f}, validation loss: {losses['val']:.3f}')

    # sample a batch of data
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.476, validation loss: 4.480
step: 250, train loss: 3.143, validation loss: 3.156
step: 500, train loss: 3.141, validation loss: 3.152
step: 750, train loss: 3.150, validation loss: 3.160
step: 1000, train loss: 3.142, validation loss: 3.151
step: 1250, train loss: 3.151, validation loss: 3.149
step: 1500, train loss: 3.140, validation loss: 3.152
step: 1750, train loss: 3.135, validation loss: 3.141
step: 2000, train loss: 3.139, validation loss: 3.155
step: 2250, train loss: 3.143, validation loss: 3.152
step: 2500, train loss: 3.142, validation loss: 3.151
step: 2750, train loss: 3.143, validation loss: 3.154
3.1042978763580322


In [8]:
prompt = 'Hello! Can you see me?'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

Hello! Can you see me?ehili o o  w
tnnunhhd   d
m
esGahehfwn pawd thcmmbnd "ss
Ca indOp kt e oViy ieuhtypoinelaadrc  otaer
